Import libraries

In [135]:
from mltrainer import ReportTypes, Trainer, TrainerSettings, metrics, rnn_models
from mltrainer.preprocessors import BasePreprocessor
from pathlib import Path
import torch
from mads_datasets import DatasetFactoryProvider, DatasetType

Setting seeds for isolated testing, but doesnt fix all randomness unfortunately?

In [ ]:
import numpy as np
import random

# Set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Get flowers data into a streamer

In [137]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor
preprocessor = BasePreprocessor()

flowers = DatasetFactoryProvider.create_factory(DatasetType.FLOWERS)
streamers = flowers.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers['train']
valid = streamers['valid']

2025-10-26 14:24:04.921 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\tycoh\.cache\mads_datasets\flowers


In [138]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64]))

Create a configurable model that can be hypertuned for the flowers dataset classification

Show you can
1. Make a hypothesis based on the theory (use the book)
1. Design experiments to test your hypothesis
1. Work iterative: eg start with a small experiment to get a direction, then reduce the search space and run a more extensive experiment

For classifying flowers we need a convolutional neural network because images are high dimensional, nearby pixels are statistically related and if pictures shifts a little al pixels values are different but it is still the same picture. By using a convolutional neural network we make sure we can use weight sharing to deal with the high dimensions, the kernel also takes care of nearby related pixels and takes care of recognizing the geomtric transformations. There are multiple architectures to choose from like LeNet, AlexNet (8 layers), VGG (19 layers), GoogLeNet (22 layers, inception), ResNet(152 layers, skip layers), SqueezeNet (less parameters, 50x less then alexnet). I am working on a simple laptop with cpu so i would like the model which is trained te fastest.   

Hypothesis
- Increasing the number of batchnorm layers increases the accuracy of the TestCNN model.


settings, trainer, ml flow logging

In [139]:
import torch.nn as nn
from typing import List

# make a CNN class
class TestCNN(nn.Module):
    # initialise class
    def __init__(self, num_classes: int, dropout: float) -> None:
        # inherent functions from module
        super().__init__()

        self.features = nn.Sequential(
                nn.Conv2d(3, 16, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2),

                nn.Conv2d(16, 32, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2),
        
                nn.Conv2d(32, 64, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2),       
        )
        self.agg = nn.AdaptiveAvgPool2d((1,1))

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Dropout(p = dropout),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
            x = self.features(x)
            x = self.agg(x)
            x = self.classifier(x)
            return x


TEST

In [155]:
import mlflow
experiment = "exercise_4"
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment)

<Experiment: artifact_location='file:c:/Users/tycoh/Desktop/MADS-ML-Tyco/4-hypertuning-ray/mlruns/2', creation_time=1761315160087, experiment_id='2', last_update_time=1761315160087, lifecycle_stage='active', name='exercise_4', tags={}>

In [156]:
mlflow.end_run()

In [153]:
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics
import torch.optim as optim

with mlflow.start_run():
    settings = TrainerSettings(
        epochs=5,
        metrics=[metrics.Accuracy()],
        logdir='modellogs',
        train_steps= 50,
        valid_steps= 50,
        reporttypes=[ReportTypes.MLFLOW]
    )
    mlflow.log_params({
        "epochs": settings.epochs,
        "metrics": settings.metrics,
        "train_steps": settings.train_steps,
        "valid_steps": settings.valid_steps
    })
    model = TestCNN(num_classes=5, dropout=0.5)

    trainer = Trainer(
        model = model,
        settings=settings,
        loss_fn=nn.CrossEntropyLoss(),
        optimizer= torch.optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau
    )
    trainer.loop()
    mlflow.end_run()

2025-10-26 15:13:01.749 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20251026-151301
2025-10-26 15:13:01.752 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 50/50 [00:27<00:00,  1.84it/s]
2025-10-26 15:13:41.031 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.5680 test 1.4668 metric ['0.3619']
100%|██████████| 50/50 [00:30<00:00,  1.64it/s]
2025-10-26 15:14:25.071 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.3551 test 1.2149 metric ['0.4713']
100%|██████████| 50/50 [00:28<00:00,  1.78it/s]
2025-10-26 15:15:05.621 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.2084 test 1.1259 metric ['0.4894']
100%|██████████| 50/50 [00:25<00:00,  1.99it/s]
2025-10-26 15:15:43.551 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.1659 test 1.0951 metric ['0.5519']
100%|██████████| 50/50 [00:27<00:00,  1.79it/s]
2025-10-26 15:16:25.6

2025-10-26 14:37:05.879 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.5653 test 1.4335 metric ['0.3764']
100%|██████████| 45/45 [00:26<00:00,  1.70it/s]
2025-10-26 14:37:35.279 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.3375 test 1.2502 metric ['0.4205']
100%|██████████| 45/45 [00:25<00:00,  1.73it/s]
2025-10-26 14:38:04.149 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.2489 test 1.1953 metric ['0.4616']

Hypothesis
- Model has an increase in accuracy by using dropout regularization because this makes the model less dependent on any given hidden unit and encourages weights to have smaller magnitudes and therefore generalize better

Experiment
- We set a seed so it is an isolated experiment with na randomness
- First, we run 3 epochs with TestCNN to examine accuracy with dropout set to 0.0
- Next, we run 3 epochs with TestCNN with dropout set to 0.2 and examine the results
- Lastly, we run 3 epochs with TestCNN with dropout set to 0.6 and examine the results

Results
- Dropout 0.0 gives max accuracy 0.48
- Dropout 0.5 gives max accuracy 0.46
- Dropout 1.0 gives max accuracy 0.21

Conclusion
- Without dropout the model has the best performance. This rejects the hypothesis that it should generalize better with dropout. I believe this is a bad experiment, because this does not match with the theory (which has been tested as well).


Possible experiments:
- timeseries with cnn
